---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-data-analysis/resources/0dhYG) course resource._

---

In [2]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind

# Assignment 4 - Hypothesis Testing
This assignment requires more individual learning than previous assignments - you are encouraged to check out the [pandas documentation](http://pandas.pydata.org/pandas-docs/stable/) to find functions or methods you might not have used yet, or ask questions on [Stack Overflow](http://stackoverflow.com/) and tag them as pandas and python related. And of course, the discussion forums are open for interaction with your peers and the course staff.

Definitions:
* A _quarter_ is a specific three month period, Q1 is January through March, Q2 is April through June, Q3 is July through September, Q4 is October through December.
* A _recession_ is defined as starting with two consecutive quarters of GDP decline, and ending with two consecutive quarters of GDP growth.
* A _recession bottom_ is the quarter within a recession which had the lowest GDP.
* A _university town_ is a city which has a high percentage of university students compared to the total population of the city.

**Hypothesis**: University towns have their mean housing prices less effected by recessions. Run a t-test to compare the ratio of the mean price of houses in university towns the quarter before the recession starts compared to the recession bottom. (`price_ratio=quarter_before_recession/recession_bottom`)

The following data files are available for this assignment:
* From the [Zillow research data site](http://www.zillow.com/research/data/) there is housing data for the United States. In particular the datafile for [all homes at a city level](http://files.zillowstatic.com/research/public/City/City_Zhvi_AllHomes.csv), ```City_Zhvi_AllHomes.csv```, has median home sale prices at a fine grained level.
* From the Wikipedia page on college towns is a list of [university towns in the United States](https://en.wikipedia.org/wiki/List_of_college_towns#College_towns_in_the_United_States) which has been copy and pasted into the file ```university_towns.txt```.
* From Bureau of Economic Analysis, US Department of Commerce, the [GDP over time](http://www.bea.gov/national/index.htm#gdp) of the United States in current dollars (use the chained value in 2009 dollars), in quarterly intervals, in the file ```gdplev.xls```. For this assignment, only look at GDP data from the first quarter of 2000 onward.

Each function in this assignment below is worth 10%, with the exception of ```run_ttest()```, which is worth 50%.

In [3]:
# Use this dictionary to map state names to two letter acronyms
states = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 'WY': 'Wyoming', 'NA': 'National', 'AL': 'Alabama', 'MD': 'Maryland', 'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 'MT': 'Montana', 'IL': 'Illinois', 'TN': 'Tennessee', 'DC': 'District of Columbia', 'VT': 'Vermont', 'ID': 'Idaho', 'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington', 'HI': 'Hawaii', 'WI': 'Wisconsin', 'MI': 'Michigan', 'IN': 'Indiana', 'NJ': 'New Jersey', 'AZ': 'Arizona', 'GU': 'Guam', 'MS': 'Mississippi', 'PR': 'Puerto Rico', 'NC': 'North Carolina', 'TX': 'Texas', 'SD': 'South Dakota', 'MP': 'Northern Mariana Islands', 'IA': 'Iowa', 'MO': 'Missouri', 'CT': 'Connecticut', 'WV': 'West Virginia', 'SC': 'South Carolina', 'LA': 'Louisiana', 'KS': 'Kansas', 'NY': 'New York', 'NE': 'Nebraska', 'OK': 'Oklahoma', 'FL': 'Florida', 'CA': 'California', 'CO': 'Colorado', 'PA': 'Pennsylvania', 'DE': 'Delaware', 'NM': 'New Mexico', 'RI': 'Rhode Island', 'MN': 'Minnesota', 'VI': 'Virgin Islands', 'NH': 'New Hampshire', 'MA': 'Massachusetts', 'GA': 'Georgia', 'ND': 'North Dakota', 'VA': 'Virginia'}

In [4]:
data = pd.read_table('university_towns.txt', header = None)
print(data.head())

                                                 0
0                                    Alabama[edit]
1                    Auburn (Auburn University)[1]
2           Florence (University of North Alabama)
3  Jacksonville (Jacksonville State University)[2]
4       Livingston (University of West Alabama)[2]


In [5]:
def get_list_of_university_towns():
    #'''Returns a DataFrame of towns and the states they are in from the 
    #university_towns.txt list. The format of the DataFrame should be:
    #DataFrame( [ ["Michigan", "Ann Arbor"], ["Michigan", "Yipsilanti"] ], 
    #columns=["State", "RegionName"]  )
    
    #The following cleaning needs to be done:

    #1. For "State", removing characters from "[" to the end.
    #2. For "RegionName", when applicable, removing every character from " (" to the end.
    #3. Depending on how you read the data, you may need to remove newline character '\n'. '''
    data = pd.read_table('university_towns.txt', header = None)
    df = []
   
    for ind, row in data.iterrows():
        item = row[0] 
        if len(item.split("(")) == 2:
            city = item.split("(")[0][:-1]
            df.append([state_key, city])
        if len(item.split("[edit")) == 2:
            state_val = item.split("[edit")[0]
            try:
                state_key = list(states.keys())[list(states.values()).index(state_val)]
            except ValueError as err:
                print(err)
    df = pd.DataFrame(df, columns=["State", "RegionName"])
    
    return df

get_list_of_university_towns()


,State,RegionName
0,AL,Auburn
1,AL,Florence
2,AL,Jacksonville
3,AL,Livingston
4,AL,Montevallo
5,AL,Troy
6,AL,Tuscaloosa
7,AL,Tuskegee
8,AK,Fairbanks
9,AZ,Flagstaff


In [6]:
GDP = pd.read_excel('gdplev.xls', skiprows=5)
GDP = GDP[['Unnamed: 4','GDP in billions of chained 2009 dollars.1']]
GDP.columns = ['Time','GDP_Value']

def myfilter(x):
    t = x[1]['Time']
    if type(t) is str and int(t.split('q')[0])>2000:
        return True
    else:
        return False

ind = list(map(lambda x: myfilter(x), GDP.iterrows()))

   
GDP = GDP[ind]

In [7]:
def get_recession_start():
    start_point = []
    for i in range(2,len(GDP)-2):
        if (GDP.iloc[i][1] < GDP.iloc[i-1][1]) and (GDP.iloc[i-1][1] < GDP.iloc[i-2][1]) and (GDP.iloc[i][1] < GDP.iloc[i+1][1]) and (GDP.iloc[i+1][1] < GDP.iloc[i+2][1]):
            start_point.append(GDP.iloc[i-2][0])
    return start_point
        
get_recession_start()


['2008q4']

In [8]:
def get_recession_end():
    end_point = []
    for i in range(2,len(GDP)-2):
        if (GDP.iloc[i][1] < GDP.iloc[i-1][1]) and (GDP.iloc[i-1][1] < GDP.iloc[i-2][1]) and (GDP.iloc[i][1] < GDP.iloc[i+1][1]) and (GDP.iloc[i+1][1] < GDP.iloc[i+2][1]):
            end_point.append(GDP.iloc[i+2][0])
    return end_point
        
get_recession_end()

['2009q4']

In [9]:
def get_recession_bottom():
    bottom_point = []
    for i in range(2,len(GDP)-2):
        if (GDP.iloc[i][1] < GDP.iloc[i-1][1]) and (GDP.iloc[i-1][1] < GDP.iloc[i-2][1]) and (GDP.iloc[i][1] < GDP.iloc[i+1][1]) and (GDP.iloc[i+1][1] < GDP.iloc[i+2][1]):
            bottom_point.append(GDP.iloc[i][0])
    return bottom_point
        
get_recession_bottom()

['2009q2']

In [10]:
zip('a','b')

In [11]:
def convert_housing_data_to_quarters():
    House = pd.read_csv('City_Zhvi_AllHomes.csv')
    House = pd.concat( [House.loc[:,['State','RegionName']], House.loc[:,'2000-01':]], axis = 1)
    House.set_index(['State','RegionName'],inplace=False)
    col = list(House.columns)

    def new_col():
        year_quarter = []
        year = list(range(2000,2017))
        quarter = ['q1','q2','q3','q4']
        for y in year:
            for q in quarter:
                if y != 2016 or q != 'q4':
                    year_quarter.append((str(y)+q))

        return year_quarter

    qs = [list(House.columns)[x:x+3] for x in range(2, len(list(House.columns)),3)]

    new_col = new_col()


    for col, q in zip(new_col, qs):
        House[col] = House[q].mean(axis=1)

    House = House[['State','RegionName']+new_col]
    House.set_index(['State','RegionName'],inplace=True)
    
    return House

convert_housing_data_to_quarters()

,,2000q1,2000q2,2000q3,2000q4,2001q1,2001q2,2001q3,2001q4,2002q1,2002q2,...,2014q2,2014q3,2014q4,2015q1,2015q2,2015q3,2015q4,2016q1,2016q2,2016q3
State,RegionName,,,,,,,,,,,,,,,,,,,,,
NY,New York,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,5.151000e+05,5.189667e+05,5.224333e+05,5.237333e+05,5.320667e+05,5.484333e+05,5.620667e+05,5.724333e+05,5.838000e+05,6.025667e+05
CA,Los Angeles,2.070667e+05,2.144667e+05,2.209667e+05,2.261667e+05,2.330000e+05,2.391000e+05,2.450667e+05,2.530333e+05,2.619667e+05,2.727000e+05,...,4.982667e+05,5.093000e+05,5.179667e+05,5.266667e+05,5.349667e+05,5.453333e+05,5.571000e+05,5.679667e+05,5.805667e+05,5.900667e+05
IL,Chicago,1.384000e+05,1.436333e+05,1.478667e+05,1.521333e+05,1.569333e+05,1.618000e+05,1.664000e+05,1.704333e+05,1.755000e+05,1.775667e+05,...,1.881333e+05,1.902333e+05,1.937333e+05,1.948000e+05,1.989333e+05,2.016333e+05,2.028667e+05,2.037333e+05,2.056000e+05,2.080000e+05
PA,Philadelphia,5.300000e+04,5.363333e+04,5.413333e+04,5.470000e+04,5.533333e+04,5.553333e+04,5.626667e+04,5.753333e+04,5.913333e+04,6.073333e+04,...,1.145667e+05,1.158667e+05,1.166000e+05,1.167333e+05,1.179333e+05,1.207000e+05,1.221667e+05,1.237000e+05,1.260000e+05,1.305667e+05
AZ,Phoenix,1.118333e+05,1.143667e+05,1.160000e+05,1.174000e+05,1.196000e+05,1.215667e+05,1.227000e+05,1.243000e+05,1.265333e+05,1.283667e+05,...,1.663000e+05,1.675333e+05,1.705000e+05,1.732667e+05,1.764667e+05,1.806000e+05,1.851333e+05,1.889000e+05,1.931000e+05,1.986333e+05
NV,Las Vegas,1.326000e+05,1.343667e+05,1.354000e+05,1.370000e+05,1.395333e+05,1.417333e+05,1.433667e+05,1.461333e+05,1.493333e+05,1.509333e+05,...,1.689667e+05,1.715667e+05,1.746333e+05,1.776000e+05,1.802000e+05,1.851667e+05,1.887667e+05,1.922333e+05,1.960333e+05,2.026667e+05
CA,San Diego,2.229000e+05,2.343667e+05,2.454333e+05,2.560333e+05,2.672000e+05,2.762667e+05,2.845000e+05,2.919333e+05,3.012333e+05,3.128667e+05,...,4.745667e+05,4.813000e+05,4.862333e+05,4.934333e+05,5.032333e+05,5.129667e+05,5.210000e+05,5.276333e+05,5.318333e+05,5.431333e+05
TX,Dallas,8.446667e+04,8.386667e+04,8.486667e+04,8.783333e+04,8.973333e+04,8.930000e+04,8.906667e+04,9.090000e+04,9.256667e+04,9.380000e+04,...,1.053000e+05,1.071667e+05,1.099333e+05,1.124667e+05,1.199667e+05,1.280667e+05,1.345333e+05,1.398000e+05,1.453000e+05,1.554333e+05
CA,San Jose,3.742667e+05,4.065667e+05,4.318667e+05,4.555000e+05,4.706667e+05,4.702000e+05,4.568000e+05,4.455667e+05,4.414333e+05,4.577667e+05,...,6.814333e+05,7.015333e+05,7.186667e+05,7.373333e+05,7.631333e+05,7.884333e+05,8.062333e+05,8.244667e+05,8.385667e+05,8.364667e+05


In [29]:
def run_ttest():

    df = get_list_of_university_towns()
    House = convert_housing_data_to_quarters()

    Resession = House.loc[:,'2008q3':'2009q2']

    Resession = Resession.reset_index()
    x = Resession['State'].isin(df['State'])
    y = Resession['RegionName'].isin(df['RegionName'])
    z=[a and b for a, b in zip(x,y)]
    Utown=Resession.loc[z]
    notz = [not zz for zz in z]
    NotUtown=Resession.loc[notz]

    def price_ratio(row):
        return (row['2008q3']-row['2009q2'])/row['2008q3']
    Utown['price_ratio'] = Utown.apply(price_ratio,axis=1)
    NotUtown['price_ratio'] = NotUtown.apply(price_ratio,axis=1)
    Utown_ratio= Utown['price_ratio'].dropna()
    NotUtown_ratio = NotUtown['price_ratio'].dropna()
    p_value = list(ttest_ind(Utown_ratio, NotUtown_ratio))[1]

    def better():
        if Utown_ratio.mean() > NotUtown_ratio.mean():
            return 'Utown has higher average house prices'
        if Utown_ratio.mean() == NotUtown_ratio.mean():
            return 'Utown and not Utown have the same average house prices'
        else:
            return 'NotUtown has higher average house prices'

    def Reject():
        if p_value < 0.01:
            return 'True'
        else:
            return 'False'

    result = (Reject(),p_value,better())

    return result

run_ttest()

C:\Users\ylin\AppData\Local\Continuum\Anaconda3\lib\site-packages\ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\ylin\AppData\Local\Continuum\Anaconda3\lib\site-packages\ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


('True', 0.0016698164996046486, 'NotUtown has higher average house prices')